# Imports & Configuration

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
spark_path = "E:\\Spark\\spark-3.2.0-bin-hadoop2.7"

In [4]:
import findspark
findspark.init(spark_path)

In [5]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

E:\Spark\spark-3.2.0-bin-hadoop2.7\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [7]:
spark

# Create the DataFrames

In [8]:
transactions_file = "E:/PySpark/data/transactions.parquet"
df_transactions = spark.read.parquet(transactions_file)

In [9]:
df_transactions.show(5, False)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+------------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt  |city        |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+------------+
|CCWG6X0ANH|2010-07-01|2019-10-01|TXCC7I6BMRX3XIT|2015-06-27|2015|6    |27 |Entertainment|35.18|denver      |
|CCWG6X0ANH|2010-07-01|2019-10-01|TGW6THZKM8C6SUF|2016-08-02|2016|8    |2  |Entertainment|58.15|new_york    |
|CCWG6X0ANH|2010-07-01|2019-10-01|TOLMMCXS90H2K31|2014-09-17|2014|9    |17 |Groceries    |43.92|boston      |
|CCWG6X0ANH|2010-07-01|2019-10-01|TXLH0P8JII96Q7U|2012-10-05|2012|10   |5  |Entertainment|35.2 |philadelphia|
|CCWG6X0ANH|2010-07-01|2019-10-01|TP6RDK6P58M2RO4|2015-10-21|2015|10   |21 |Entertainment|47.44|chicago     |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+-----+------------+
only showi

In [10]:
customers_file = "E:/PySpark/data/customers.parquet"
df_customers = spark.read.parquet(customers_file)

In [11]:
df_customers.show(5, False)

+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows



# Spark's Query Plans

# Narrow Transformations
- `filter` rows where `city='boston'`
- `add` a new column: adding `first_name` and `last_name`
- `alter` an exisitng column: adding 5 to `age` column
- `select` relevant columns

In [12]:
df_narrow_transform = (
    df_customers
    .filter(F.col("city") == "boston")
    .withColumn("first_name", F.split("name", " ").getItem(0))
    .withColumn("last_name", F.split("name", " ").getItem(1))
    .withColumn("age", F.col("age") + F.lit(5))
    .select("cust_id", "first_name", "last_name", "age", "gender", "birthday")
)

df_narrow_transform.show(5, False)

+----------+----------+---------+----+------+---------+
|cust_id   |first_name|last_name|age |gender|birthday |
+----------+----------+---------+----+------+---------+
|C007YEYTX9|Aaron     |Abbott   |39.0|Female|7/13/1991|
|C08XAQUY73|Aaron     |Lambert  |59.0|Female|11/5/1966|
|C094P1VXF9|Aaron     |Lindsey  |29.0|Male  |9/21/1990|
|C097SHE1EF|Aaron     |Lopez    |27.0|Female|4/18/2001|
|C0DTC6436T|Aaron     |Schwartz |57.0|Female|7/9/1962 |
+----------+----------+---------+----+------+---------+
only showing top 5 rows



In [14]:
df_narrow_transform.explain(True)

== Parsed Logical Plan ==
'Project ['cust_id, 'first_name, 'last_name, 'age, 'gender, 'birthday]
+- Project [cust_id#67, name#68, (cast(age#69 as double) + cast(5 as double)) AS age#129, gender#70, birthday#71, zip#72, city#73, first_name#110, last_name#119]
   +- Project [cust_id#67, name#68, age#69, gender#70, birthday#71, zip#72, city#73, first_name#110, split(name#68,  , -1)[1] AS last_name#119]
      +- Project [cust_id#67, name#68, age#69, gender#70, birthday#71, zip#72, city#73, split(name#68,  , -1)[0] AS first_name#110]
         +- Filter (city#73 = boston)
            +- Relation [cust_id#67,name#68,age#69,gender#70,birthday#71,zip#72,city#73] parquet

== Analyzed Logical Plan ==
cust_id: string, first_name: string, last_name: string, age: double, gender: string, birthday: string
Project [cust_id#67, first_name#110, last_name#119, age#129, gender#70, birthday#71]
+- Project [cust_id#67, name#68, (cast(age#69 as double) + cast(5 as double)) AS age#129, gender#70, birthday#71, 

# Wide Transformations
1. Repartition
2. Coalesce
3. Joins
4. GroupBy
   - `count`
   - `countDistinct`
   - `sum`

## 1. Repartition

In [15]:
df_transactions.rdd.getNumPartitions()

8

In [16]:
df_transactions.repartition(24).explain(True)

== Parsed Logical Plan ==
Repartition 24, true
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 24, true
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Optimized Logical Plan ==
Repartition 24, true
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(24), REPARTITION_BY_NUM, [id=#72]
   +- FileScan parquet [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] Batched: true, DataFilters: [], Format: Parquet, Location: I

In [19]:
df_transactions.repartition(24).show(10)

+----------+----------+----------+---------------+----------+----+-----+---+-------------------+------+------------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day|       expense_type|   amt|        city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------------+------+------------+
|C0YDPQWPBJ|2010-04-01|2019-09-01|TVXUVPGJMNDVJNT|2016-10-09|2016|   10|  9|Bills and Utilities|260.43|philadelphia|
|CGC6V1LCDF|2010-07-01|2020-03-01|TIY34BZINOGEBR1|2015-06-27|2015|    6| 27|      Entertainment| 16.25|      boston|
|C0YDPQWPBJ|2011-06-01|2020-07-01|TO4XD5EU6MFQ7XU|2017-06-07|2017|    6|  7|      Entertainment|  9.02|     seattle|
|C0YDPQWPBJ|2011-12-01|      null|T8J5TXH2ZNQZ1N8|2016-02-05|2016|    2|  5|       Motor/Travel| 30.07|philadelphia|
|C0YDPQWPBJ|2011-09-01|2020-04-01|TFA4WZVS391HZ5R|2017-07-26|2017|    7| 26|      Entertainment| 17.64|    new_york|
|C0YDPQWPBJ|2011-06-01|2020-07-01|TMQIXF7Q05THK6Z|2019-06-05|201

## 2. Coalesce

In [17]:
df_transactions.coalesce(5).explain(True)

== Parsed Logical Plan ==
Repartition 5, false
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 5, false
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Optimized Logical Plan ==
Repartition 5, false
+- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet

== Physical Plan ==
Coalesce 5
+- *(1) ColumnarToRow
   +- FileScan parquet [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/E:/PySpark/data/transactions.parquet], P

In [18]:
df_transactions.coalesce(5).show(10)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+------------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day| expense_type|   amt|        city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+------------+
|CCWG6X0ANH|2010-07-01|2019-10-01|TXCC7I6BMRX3XIT|2015-06-27|2015|    6| 27|Entertainment| 35.18|      denver|
|CCWG6X0ANH|2010-07-01|2019-10-01|TGW6THZKM8C6SUF|2016-08-02|2016|    8|  2|Entertainment| 58.15|    new_york|
|CCWG6X0ANH|2010-07-01|2019-10-01|TOLMMCXS90H2K31|2014-09-17|2014|    9| 17|    Groceries| 43.92|      boston|
|CCWG6X0ANH|2010-07-01|2019-10-01|TXLH0P8JII96Q7U|2012-10-05|2012|   10|  5|Entertainment|  35.2|philadelphia|
|CCWG6X0ANH|2010-07-01|2019-10-01|TP6RDK6P58M2RO4|2015-10-21|2015|   10| 21|Entertainment| 47.44|     chicago|
|CCWG6X0ANH|2010-07-01|2019-10-01|TCTF14CSG65AEFW|2018-07-18|2018|    7| 18|Entertainment| 29.67| los_angeles|
|

### Why doesn't `.coalesce()` explicitly show the partitioning scheme?

`.coalesce` doesn't show the partitioning scheme e.g. `RoundRobinPartitioning` because: 
- The operation only minimizes data movement by merging into fewer partitions, it doesn't do any shuffling.
- Because no shuffling is done, the partitioning scheme remains the same as the original DataFrame and Spark doesn't include it explicitly in it's plan as the partitioning scheme is unaffected by `.coalesce`

## 3. Joins (Shuffle Sort Merge Join)

In [22]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'-1'

In [21]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [23]:
df_joined = (
    df_transactions.join(
        df_customers,
        how="inner",
        on="cust_id"
    )
)

In [24]:
df_joined.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(cust_id))
:- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet
+- Relation [cust_id#67,name#68,age#69,gender#70,birthday#71,zip#72,city#73] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string, name: string, age: string, gender: string, birthday: string, zip: string, city: string
Project [cust_id#0, start_date#1, end_date#2, txn_id#3, date#4, year#5, month#6, day#7, expense_type#8, amt#9, city#10, name#68, age#69, gender#70, birthday#71, zip#72, city#73]
+- Join Inner, (cust_id#0 = cust_id#67)
   :- Relation [cust_id#0,start_date#1,end_date#2,txn_id#3,date#4,year#5,month#6,day#7,expense_type#8,amt#9,city#10] parquet
   +- Relation [cust_id#67,name#68,age#69,gender#70,birthday#71,zip#72,city#73] parquet

== O

In [25]:
df_joined.show()

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-------------+----------+---+------+---------+-----+--------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day| expense_type|   amt|         city|      name|age|gender| birthday|  zip|    city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-------------+----------+---+------+---------+-----+--------+
|C0Y61QV84S|2011-01-01|2020-03-01|TNQ5HWKS4IJ0ADX|2011-07-04|2011|    7|  4|Entertainment| 11.16|san_francisco|Ada Keller| 26|Female|6/25/1961|77925|portland|
|C0Y61QV84S|2011-01-01|2020-03-01|T500LLXZ7S5SU5J|2019-07-01|2019|    7|  1|          Tax|125.19|     new_york|Ada Keller| 26|Female|6/25/1961|77925|portland|
|C0Y61QV84S|2011-01-01|2020-03-01|TNS0XYGHCDB68KM|2017-05-04|2017|    5|  4|Entertainment| 23.41|  los_angeles|Ada Keller| 26|Female|6/25/1961|77925|portland|
|C0Y61QV84S|2011-01-01|2020-03-01|TLZVV9F60FFI

## 4. GroupBy

In [ ]:
df_transactions.printSchema()

### GroupBy Count

In [ ]:
df_city_counts = (
    df_transactions
    .groupBy("city")
    .count()
)

In [ ]:
df_city_counts.explain(True)

In [ ]:
df_txn_amt_city = (
    df_transactions
    .groupBy("city")
    .agg(F.sum("amt").alias("txn_amt"))
)

In [ ]:
df_txn_amt_city.explain(True)

### GroupBy - countDistinct

In [ ]:
df_txn_per_city = (
    df_transactions
    .groupBy("cust_id")
    .agg(F.countDistinct("city").alias("city_count"))
)

In [ ]:
df_txn_per_city.show(5, False)
df_txn_per_city.explain(True)

# 5. Some Observations

### Why is a filter step present despite predicate pushdown? 

This is largely due to the way `Spark's Catalyst Optimizer` works. Specifically, due to two separate stages of the query optimization process: Physical Planning and Logical Planning.

- **Logical Planning**: Catalyst optimizer simplifies the unresolved logical plan (which represents the user's query) by applying various rule-based optimizations. This includes `predicate pushdown`, `projection pushdown` where filter conditions and column projections are moved as close to the data source as possible.

- **Physical Planning** phase is where the logical plan is translated into one or more physical plans, which can actually be executed on the cluster. This includes operations like file `scans`, `filters`, `projections`, etc.

In this case, during the logical planning phase, the predicate (`F.col("city") == "boston"`) has been pushed down and will be applied during the scan of the Parquet file (`PushedFilters: [IsNotNull(city), EqualTo(city,boston)]`), thus improving performance.

Now, during the physical planning phase, the same filter condition (`+- *(1) Filter (isnotnull(city#73) AND (city#73 = boston))`) is applied again to the data that's been loaded into memory. This is because of the following reasons:

1. **Guaranteed Correctness:** It might seem **redundant**, but remember that not all data sources can handle pushed-down predicates, and not all predicates can be pushed down. Therefore, **even if a predicate is pushed down to the data source, Spark still includes the predicate in the physical plan** to cover cases where the data source might not have been able to fully apply the predicate. This is Spark's way of making sure the correct data is always returned, no matter the capabilities of the data source.

2. **No Assumptions**: Spark's Catalyst optimizer doesn't make assumptions about the data source's ability to handle pushed-down predicates. The optimizer aims to generate plans that return correct results across a wide range of scenarios. Even if the filter is pushed down, Spark does not have the feedback from data source whether the pushdown was successful or not, so it includes the filter operation in the physical plan as well.

It is more of a **fail-safe mechanism** to ensure data **integrity** and **correctness**.

---

### In what cases will predicate pushdown not work?

2 Examples where **filter pushdown** will not work:

1. **Complex Data Types**: Spark's Parquet data source does not push down filters that involve **complex types**, such as **arrays**, **maps**, and **structs**. This is because these complex data types can have complicated nested structures that the Parquet reader cannot easily filter on.

Here's an example:

```
root
 |-- Name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|Name      |properties                   |
+----------+-----------------------------+
|Afaque    |[eye -> black, hair -> black]|
|Naved     |[eye ->, hair -> brown]      |
|Ali       |[eye -> black, hair -> red]  |
|Amaan     |[eye -> grey, hair -> grey]  |
|Omaira    |[eye -> , hair -> brown]     |
+----------+-----------------------------+
```

```python
df.filter(df.properties.getItem("eye") == "brown").show()
```

```
== Physical Plan ==
*(1) Filter (metadata#123[key] = value)
+- *(1) ColumnarToRow
   +- FileScan parquet [id#122,metadata#123] Batched: true, DataFilters: [(metadata#123[key] = value)], Format: Parquet, ...
```

------------------------------------------------

3. Unsupported Expressions: 

In Spark, `Parquet` data source does not support pushdown for filters involving a `.cast` operation. The reason for this behaviour is as follows:
- `.cast` changes the datatype of the column, and the Parquet data source may not be able to perform the filter operation correctly on the cast data.

**Note**: This behavior may vary based on the data source. For example, if you're working with a JDBC data source connected to a database that supports SQL-like operations, the `.cast` filter could potentially be pushed down to the database.

### Example of operation where filter pushdown doesn't work

In [ ]:
df_customer_gt_50 = (
    df_customers
    .filter(F.col("age").cast("int") > 50)
)
df_customer_gt_50.show(5, False)
df_customer_gt_50.explain(True)

In [ ]:
spark.stop()